# Connecting with ThingSpeak

### The HW

<p><img src="https://github.com/Mjrovai/Python4DS/blob/master/Micropython/IoT_TS_MQTT/electrDiagr.png?raw=true"> </p>




### Connect with ESP8266 via serial port

In [1]:
%serialconnect to --port=/dev/tty.SLAB_USBtoUART --baud=115200

Connecting to --port=/dev/tty.SLAB_USBtoUART --baud=115200 
Ready.


### DHT Sensor

In [2]:
from dht import DHT22
from machine import Pin
dht22 = DHT22(Pin(12))

In [3]:
# Function to read DHT
def readDht():
    dht22.measure()
    return dht22.temperature(), dht22.humidity()

In [4]:
# test DHT function
print (readDht())

(19.1, 37.0)


### DS18B20 sensor

In [5]:
# Import libraries 
import onewire, ds18x20
import time

In [6]:
# Define which pin the 1-wire device will be connected ==> pin 2 (D4)
dat = Pin(2)

In [7]:
# create the onewire object
ds = ds18x20.DS18X20(onewire.OneWire(dat))

In [8]:
# scan for devices on the bus
sensors = ds.scan()
print('found devices:', sensors)

found devices: [bytearray(b'(\xff\xd3\xe6a\x16\x041')]


In [9]:
# function to read DS18B20 
def readDs(): 
    ds.convert_temp()
    time.sleep_ms(750)
    return round(ds.read_temp(sensors[0]), 1)

In [10]:
# test readDs function
print(readDs())

17.9


### LDR Sensor

In [11]:
# import library
from machine import ADC

In [12]:
# Define object
adc = ADC(0)

In [13]:
print(adc.read())

667


In [14]:
#function to read luminosity
def readLdr():
    lumPerct = (adc.read()-40)*(10/86) # convert in percentage ("map")
    return round(lumPerct)

In [15]:
# test LDR read function
print (readLdr())

73


### Button (Digital Input)

In [16]:
# define pin 13 as an input and activate an internal Pull-up resistor:
button = Pin(13, Pin.IN, Pin.PULL_UP)

In [17]:
# Function to read button state:
def readBut():
        return button.value()

In [18]:
# test button read function
print (readBut())

1


### Define Function to collect sensor data

In [19]:
def colectData():
    temp, hum, = readDht()
    extTemp = readDs()
    lum = readLdr()
    butSts = readBut()
    return temp, hum, extTemp, lum, butSts

In [20]:
# test function
print(colectData())

(18.1, 39.1, 17.9, 73, 1)


### Displaying Data on OLED

In [21]:
# import library and create object i2c
from machine import I2C
i2c = I2C(scl=Pin(5), sda=Pin(4))

In [22]:
# import library and create object oled
import ssd1306
i2c = I2C(scl=Pin(5), sda=Pin(4))
oled = ssd1306.SSD1306_I2C(128, 64, i2c, 0x3c)

In [23]:
# test OLED
oled.fill(0)
oled.text("Hello esp8266", 0, 0)
oled.show()

In [24]:
# Read Sensors
temp, hum, extTemp, lum, butSts = colectData()

In [25]:
# Show data on display once 
oled.fill(0)
oled.text("Temp:    " + str(temp) + "oC", 0, 4)
oled.text("Hum:     " + str(hum) + "%",0, 16)
oled.text("ExtTemp: " + str(extTemp) + "oC", 0, 29)
oled.text("Lumin:   " + str(lum) + "%", 0, 43)
oled.text("Button:  " + str(butSts), 0, 57)
oled.show()

In [26]:
# create a function to display data:
def displayData(temp, hum, extTemp, lum, butSts):
    oled.fill(0)
    oled.text("Temp:    " + str(temp) + "oC", 0, 4)
    oled.text("Hum:     " + str(hum) + "%",0, 16)
    oled.text("ExtTemp: " + str(extTemp) + "oC", 0, 29)
    oled.text("Lumin:   " + str(lum) + "%", 0, 43)
    oled.text("Button:  " + str(butSts), 0, 57)
    oled.show()

In [27]:
# define pin 0 as output
led = Pin(0, Pin.OUT)

In [28]:
# display data with a function
led.on()
temp, hum, extTemp, lum, butSts = colectData()
displayData(temp, hum, extTemp, lum, butSts)
led.off()

In [29]:
# Clear display :
def displayClear():
    oled.fill(0)
    oled.show()

In [30]:
from time import sleep

In [31]:
# create a blink function
def blinkLed(num):
    for i in range(0, num):
        led.on()
        sleep(0.5)
        led.off()
        sleep(0.5)

In [32]:
PUB_TIME_SEC = 10

In [33]:
# loop getting data until button is pressed
while button.value():
        led.on()
        temp, hum, extTemp, lum, butSts = colectData()
        displayData(temp, hum, extTemp, lum, butSts)
        led.off()
        time.sleep(PUB_TIME_SEC)
blinkLed(3)
displayClear()

.....

### Connect with WiFi

In [34]:
import network
WiFi_SSID = "YOUR SSID"
WiFi_PASS = "YOUR PASSWORD"

In [35]:
# Function to connect to local WiFi
def do_connect():
    wlan = network.WLAN(network.STA_IF)
    wlan.active(True)
    if not wlan.isconnected():
        print('connecting to network...')
        wlan.connect(WiFi_SSID, WiFi_SSID)
        while not wlan.isconnected():
            pass
    print('network config:', wlan.ifconfig())

In [36]:
# connect to local WiFi
do_connect()

network config: ('10.0.1.2', '255.255.255.0', '10.0.1.1', '10.0.1.1')


## Connect with ThingSpeak via MTQQ

MTTQ protocol code, based on examples created by Mike Teachman:
<p>https://github.com/MikeTeachman/micropython-thingspeak-mqtt-esp8266</p>

The MQTT protocol is supported in a built-in library in the Micropython binaries -- this protocol can be used send data from your ESP8266, over WIFI, to a free cloud database

### Configurate & Initiate ThingSpeak Channel and MQTT 

<p><img src="https://blogs.mathworks.com/iot/files/2017/01/ThingSpeak_MQTT_Broker.png?raw=true"></p>


In [37]:
from umqtt.simple import MQTTClient

In [38]:
# ThingSpeak Credentials:
SERVER = "mqtt.thingspeak.com"
CHANNEL_ID = "YOUR CHANNEL ID"
WRITE_API_KEY = "YOUR API KEY"
PUB_TIME_SEC = 30  

In [39]:
client = MQTTClient("umqtt_client", SERVER)

In [40]:
# Create the topic string
topic = "channels/" + CHANNEL_ID + "/publish/" + WRITE_API_KEY

In [41]:
# Read Sensors
temp, hum, extTemp, lum, butSts = colectData()

In [42]:
# Create a payload based on your Channel fields
payload = "field1="+str(temp)+"&field2="+str(hum)+"&field3="+str(extTemp)+"&field4="+str(lum)+"&field5="+str(butSts)

In [43]:
# Sending data to ThingSpeak one time for test 
client.connect()
client.publish(topic, payload)
client.disconnect() 

---

### Logging sensor data on ThingSpeak

In [44]:
# Interrupt cell with "Interrupt" on Kernel Menu ([Ctrl]+[C]) to stop sending data 
while True:
    temp, hum, extTemp, lum, butSts = colectData()
    payload = "field1="+str(temp)+"&field2="+str(hum)+"&field3="+str(extTemp)+"&field4="+str(lum)+"&field5="+str(butSts)
    client.connect()
    client.publish(topic, payload)
    client.disconnect()
    time.sleep(PUB_TIME_SEC)

.........

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 8, in <module>
KeyboardInterrupt: 


In [45]:
# loop getting data until button is pressed
while button.value():
        led.on()
        temp, hum, extTemp, lum, butSts = colectData()
        displayData(temp, hum, extTemp, lum, butSts)
        led.off()
        payload = "field1="+str(temp)+"&field2="+str(hum)+"&field3="+str(extTemp)+"&field4="+str(lum)+"&field5="+str(butSts)
        client.connect()
        client.publish(topic, payload)
        client.disconnect()
        
        time.sleep(PUB_TIME_SEC)
blinkLed(3)
displayClear()

.................................................................